In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.optim as optim

In [2]:
# For M
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# For M' and M'' and M*
class Net_sub(nn.Module):
    def __init__(self):
        super(Net_sub, self).__init__()
        self.fc1 = nn.Linear(784, 64) # subbed from gaussian
        self.fc2 = nn.Linear(64, 64)  # changed to match dimension
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
  

In [3]:
M = Net()
M.load_state_dict(torch.load('model.pth'))
M.eval()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

### Creating $S_{64\times128}$

In [4]:
shape = (64,128)
S = np.random.normal(0, 1, shape)
S = torch.tensor(S, dtype=torch.float32)

shape2 = (64,128) # or S2 = S
S2 = np.random.normal(0, 1, shape2)
S2 = torch.tensor(S2, dtype=torch.float32)

In [5]:
M.fc2.weight.data.shape

torch.Size([64, 128])

In [6]:
print(S2.shape, M.fc2.bias.data.shape)

torch.Size([64, 128]) torch.Size([64])


In [7]:
A_dash_weights_L1 = S @ M.fc1.weight.data.clone()
A_dash_bias_L1 = S @ M.fc1.bias.data.clone()

A_dash_weights_L2 = S2 @ (M.fc2.weight.data.clone()).T
A_dash_bias_L2 = S2.T @ (M.fc2.bias.data.clone())

In [8]:
print(M.fc2.weight.data.shape)

torch.Size([64, 128])


In [9]:
print(A_dash_weights_L1.shape, A_dash_bias_L1.shape)
print(A_dash_weights_L2.shape, A_dash_bias_L2.shape)

torch.Size([64, 784]) torch.Size([64])
torch.Size([64, 64]) torch.Size([128])


In [10]:
print(M.fc2.bias.data.shape)

torch.Size([64])


In [11]:
M_dash = Net_sub()

M_dash.fc1.weight.data = A_dash_weights_L1
M_dash.fc1.bias.data = A_dash_bias_L1

M_dash.fc2.weight.data = A_dash_weights_L2
M_dash.fc2.bias.data = M.fc2.bias.data.clone()

M_dash.fc3.weight.data = M.fc3.weight.data.clone()
M_dash.fc3.bias.data = M.fc3.bias.data.clone()


In [12]:
print(M_dash.fc1.weight.data.shape, M_dash.fc1.bias.data.shape)
print(M_dash.fc2.weight.data.shape, M_dash.fc2.bias.data.shape)

torch.Size([64, 784]) torch.Size([64])
torch.Size([64, 64]) torch.Size([64])


In [13]:
M_dash.eval()

Net_sub(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [14]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [15]:
M_dash.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = M_dash(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total

print(f'Test Accuracy model: {accuracy}%')

Test Accuracy model: 7.76%


In [16]:
import copy
M_double_dash = copy.deepcopy(M_dash)

In [17]:
M_double_dash

Net_sub(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [18]:
# Train the model M'' on the MNIST dataset
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(M_double_dash.parameters(), lr=0.01)
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = M_double_dash(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 1.264894723892212
Epoch 2/10, Loss: 0.38062405586242676
Epoch 3/10, Loss: 0.44410866498947144
Epoch 4/10, Loss: 0.3563108742237091
Epoch 5/10, Loss: 0.5074821710586548
Epoch 6/10, Loss: 0.4351373016834259
Epoch 7/10, Loss: 0.12348975986242294
Epoch 8/10, Loss: 0.28328076004981995
Epoch 9/10, Loss: 0.20723871886730194
Epoch 10/10, Loss: 0.10706549882888794


In [19]:
M_double_dash.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = M_double_dash(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total

print(f'Test Accuracy model: {accuracy}%')

Test Accuracy model: 93.34%


In [20]:
M_star = Net_sub()
# Train the model M* on the MNIST dataset
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(M_star.parameters(), lr=0.01)
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = M_star(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


Epoch 1/10, Loss: 0.4775153696537018
Epoch 2/10, Loss: 0.27576935291290283
Epoch 3/10, Loss: 0.23371675610542297
Epoch 4/10, Loss: 0.0607939213514328
Epoch 5/10, Loss: 0.05723539739847183
Epoch 6/10, Loss: 0.0828898623585701
Epoch 7/10, Loss: 0.052104827016592026
Epoch 8/10, Loss: 0.04777601361274719
Epoch 9/10, Loss: 0.019970186054706573
Epoch 10/10, Loss: 0.24144688248634338


In [21]:
# Evaluate the model M* on the test set
M_star.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = M_star(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')


Test Accuracy: 96.02%
